In [0]:
import azureml.core
from azureml.core import Workspace
from azureml.core import Datastore
from azureml.core import Dataset
from azureml.data.datapath import DataPath


# check core SDK version number
print(f'Azure ML SDK Version: {azureml.core.VERSION}')

Azure ML SDK Version: 1.37.0

In [0]:
%sh pip freeze | grep azureml

azureml-automl-core==1.37.0
azureml-core==1.37.0.post1
azureml-dataprep==2.25.2
azureml-dataprep-native==38.0.0
azureml-dataprep-rslex==2.1.1
azureml-dataset-runtime==1.37.0
azureml-mlflow==1.37.0
azureml-pipeline==1.37.0
azureml-pipeline-core==1.37.0
azureml-pipeline-steps==1.37.0
azureml-sdk==1.37.0
azureml-telemetry==1.37.0
azureml-train-automl-client==1.37.0
azureml-train-core==1.37.0
azureml-train-restclients-hyperdrive==1.37.0

In [0]:
workspace_name = 'mlsalondata'
subscription_id = 'f80606e5-788f-4dc3-a9ea-2eb9a7836082'
resource_group_name = 'rg-salon-data'
tenant_id = '8e2e7c2d-4702-496d-af6c-96e4bfc9f667'

In [0]:
from azureml.core.authentication import InteractiveLoginAuthentication

interactive_auth = InteractiveLoginAuthentication(tenant_id=tenant_id)

In [0]:
# connect to Azure ML
ws = Workspace(workspace_name = workspace_name,
            subscription_id = subscription_id,
            resource_group = resource_group_name,
            auth=interactive_auth)

In [0]:
ws.get_details()

Out[6]: {'id': '/subscriptions/f80606e5-788f-4dc3-a9ea-2eb9a7836082/resourceGroups/rg-salon-data/providers/Microsoft.MachineLearningServices/workspaces/mlsalondata',
 'name': 'mlsalondata',
 'identity': {'principal_id': '2da1c65a-4ba3-4abd-8720-94cf3748c325',
 'tenant_id': '8e2e7c2d-4702-496d-af6c-96e4bfc9f667',
 'type': 'SystemAssigned'},
 'location': 'francecentral',
 'type': 'Microsoft.MachineLearningServices/workspaces',
 'tags': {},
 'sku': 'Basic',
 'workspaceid': 'f5638aa7-c86d-482e-b2c6-52a6f0616610',
 'sdkTelemetryAppInsightsKey': '0cbbdb9b-2024-4898-b838-d1bff12d58e7',
 'description': '',
 'friendlyName': 'mlsalondata',
 'creationTime': '2021-09-22T08:55:00.4248614+00:00',
 'containerRegistry': '/subscriptions/f80606e5-788f-4dc3-a9ea-2eb9a7836082/resourcegroups/rg-salon-data/providers/microsoft.containerregistry/registries/crsalondata',
 'adbWorkspace': '/subscriptions/f80606e5-788f-4dc3-a9ea-2eb9a7836082/resourceGroups/rg-salon-data/providers/Microsoft.Databricks/workspaces/dbxsalondata francecentral',
 'keyVault': '/subscriptions/f80606e5-788f-4dc3-a9ea-2eb9a7836082/resourcegroups/rg-salon-data/providers/microsoft.keyvault/vaults/kvsalondata',
 'applicationInsights': '/subscriptions/f80606e5-788f-4dc3-a9ea-2eb9a7836082/resourcegroups/rg-salon-data/providers/microsoft.insights/components/appisalondata',
 'storageAccount': '/subscriptions/f80606e5-788f-4dc3-a9ea-2eb9a7836082/resourcegroups/rg-salon-data/providers/microsoft.storage/storageaccounts/samlsalondata',
 'hbiWorkspace': False,
 'allowPublicAccessWhenBehindVnet': False,
 'provisioningState': 'Succeeded',
 'discoveryUrl': 'https://francecentral.api.azureml.ms/discovery',
 'notebookInfo': {'fqdn': 'ml-mlsalonda-francecentral-f5638aa7-c86d-482e-b2c6-52a6f0616610.notebooks.azure.net',
 'resource_id': 'a4a0cb3c8867430cbb2607230eed70fc'}}

In [0]:
import mlflow
import mlflow.azureml
import azureml.mlflow
import azureml.core

import pandas as pd
import numpy as np

In [0]:
uri = ws.get_mlflow_tracking_uri()
mlflow.set_tracking_uri(uri)

In [0]:
print(f'MLFlow tracking URI is {uri}')

MLFlow tracking URI is azureml://francecentral.api.azureml.ms/mlflow/v1.0/subscriptions/f80606e5-788f-4dc3-a9ea-2eb9a7836082/resourceGroups/rg-salon-data/providers/Microsoft.MachineLearningServices/workspaces/mlsalondata?

In [0]:
#Set MLflow experiment. 
#experimentName = "/Users/paul.peton@live.fr/mlflows/wine_experiment"
experimentName = "wine_aml_experiment" 
mlflow.set_experiment(experimentName) 

Out[10]: <Experiment: artifact_location='', experiment_id='a634ccea-ddcf-414c-b84e-616655970876', lifecycle_stage='active', name='wine_aml_experiment', tags={}>

https://databricks.com/notebooks/gallery/MLEndToEndExampleAWS.html

In [0]:
white_wine = pd.read_csv('/dbfs/databricks-datasets/wine-quality/winequality-white.csv', sep=';')
red_wine = pd.read_csv('/dbfs/databricks-datasets/wine-quality/winequality-red.csv', sep=';')

In [0]:
display(white_wine)

fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.001,3.0,0.45,8.8,6
6.3,0.3,0.34,1.6,0.049,14.0,132.0,0.994,3.3,0.49,9.5,6
8.1,0.28,0.4,6.9,0.05,30.0,97.0,0.9951,3.26,0.44,10.1,6
7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.4,9.9,6
7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.4,9.9,6
8.1,0.28,0.4,6.9,0.05,30.0,97.0,0.9951,3.26,0.44,10.1,6
6.2,0.32,0.16,7.0,0.045,30.0,136.0,0.9949,3.18,0.47,9.6,6
7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.001,3.0,0.45,8.8,6
6.3,0.3,0.34,1.6,0.049,14.0,132.0,0.994,3.3,0.49,9.5,6
8.1,0.22,0.43,1.5,0.044,28.0,129.0,0.9938,3.22,0.45,11.0,6


In [0]:
red_wine['is_red'] = 1
white_wine['is_red'] = 0
 
data = pd.concat([red_wine, white_wine], axis=0)
 
# Remove spaces from column names
data.rename(columns=lambda x: x.replace(' ', '_'), inplace=True)

In [0]:
data.isna().any()

Out[14]: fixed_acidity False
volatile_acidity False
citric_acid False
residual_sugar False
chlorides False
free_sulfur_dioxide False
total_sulfur_dioxide False
density False
pH False
sulphates False
alcohol False
quality False
is_red False
dtype: bool

In [0]:
high_quality = (data.quality >= 7).astype(int)
data.quality = high_quality

In [0]:
from sklearn.model_selection import train_test_split
 
train, test = train_test_split(data, random_state=123)
X_train = train.drop(["quality"], axis=1)
X_test = test.drop(["quality"], axis=1)
y_train = train.quality
y_test = test.quality

In [0]:
import mlflow.pyfunc
import mlflow.sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from mlflow.models.signature import infer_signature
 
# The predict method of sklearn's RandomForestClassifier returns a binary classification (0 or 1). 
# The following code creates a wrapper function, SklearnModelWrapper, that uses 
# the predict_proba method to return the probability that the observation belongs to each class. 
 
class SklearnModelWrapper(mlflow.pyfunc.PythonModel):
  def __init__(self, model):
    self.model = model
    
  def predict(self, context, model_input):
    return self.model.predict_proba(model_input)[:,1]
 
# mlflow.start_run creates a new MLflow run to track the performance of this model. 
# Within the context, you call mlflow.log_param to keep track of the parameters used, and
# mlflow.log_metric to record metrics like accuracy.
with mlflow.start_run(run_name='wine_rf_run') as run:
  
  runID = run.info.run_uuid
  experimentID = run.info.experiment_id
  print(f'This is the run {runID} in the experiment {experimentID}')
  
  n_estimators = 10
  model = RandomForestClassifier(n_estimators=n_estimators, random_state=np.random.RandomState(42))
  model.fit(X_train, y_train)
  
  #mlflow.sklearn.autolog()
 
  # predict_proba returns [prob_negative, prob_positive], so slice the output with [:, 1]
  predictions_test = model.predict_proba(X_test)[:,1]
  auc_score = roc_auc_score(y_test, predictions_test)
  mlflow.log_param('n_estimators', n_estimators)
  # Use the area under the ROC curve as a metric.
  mlflow.log_metric('auc', auc_score)
  wrappedModel = SklearnModelWrapper(model)
  # Log the model with a signature that defines the schema of the model's inputs and outputs. 
  # When the model is deployed, this signature will be used to validate inputs.
  signature = infer_signature(X_train, wrappedModel.predict(None, X_train))

  mlflow.pyfunc.log_model("wine_rf_model", python_model=wrappedModel, signature=signature)
  
  #run.register_model(model_name = 'wine_rf_model', model_path = 'model')
  # !!! 'ActiveRun' object has no attribute 'register_model'

This is the run 5371ec0c-7078-40e1-a418-40c2e4185b3b in the experiment a634ccea-ddcf-414c-b84e-616655970876
/databricks/python/lib/python3.8/site-packages/mlflow/models/signature.py:129: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
 inputs = _infer_schema(model_input)

In [0]:
feature_importances = pd.DataFrame(model.feature_importances_, index=X_train.columns.tolist(), columns=['importance'])
feature_importances.sort_values('importance', ascending=False)

Out[21]:

,importance
alcohol,0.143085
density,0.120277
chlorides,0.107933
volatile_acidity,0.095391
sulphates,0.082023
residual_sugar,0.079612
total_sulfur_dioxide,0.079440
pH,0.076515
citric_acid,0.075548
free_sulfur_dioxide,0.071595


In [0]:
all_runs = mlflow.search_runs(max_results=10)  # Note : This is pandas dataframe
display(all_runs)

In [0]:
child_runs = all_runs.dropna(subset=["metrics.auc"])
display(child_runs[["status","params.n_estimators","metrics.auc"]])

In [0]:
%fs ls /mnt/gold/models/

https://tsmatz.github.io/azure-databricks-exercise/exercise10-mlflow.html

https://docs.microsoft.com/en-us/azure/machine-learning/how-to-deploy-mlflow-models

In [0]:
#https://www.mlflow.org/docs/latest/python_api/mlflow.azureml.html

import mlflow.azureml
from azureml.core.webservice import AciWebservice, Webservice

registered_image, registered_model = mlflow.azureml.build_image(
  model_uri="/mnt/gold/models/wine_rf_model",
  image_name="wine_predict_image",
  model_name="wine_rf_model",
  workspace=ws,
  synchronous=True)

# If your image build failed, you can access build logs at the following URI:
print("Access the following URI for build logs: {}".format(registered_image.image_build_log_uri))

# Deploy the image to Azure Container Instances (ACI) for real-time serving
webservice_deployment_config = AciWebservice.deploy_configuration()
webservice = Webservice.deploy_from_image(
                    image=azure_image, workspace=azure_workspace, name="wine-predict-aci")
webservice.wait_for_deployment()

<command-3698963280378827>:4: DeprecationWarning: ``mlflow.azureml.build_image`` is deprecated since 1.19.0. This method will be removed in a near future release. Use ``the azureml deployment plugin, https://aka.ms/aml-mlflow-deploy`` instead.
 registered_image, registered_model = mlflow.azureml.build_image(
Registering model wine_rf_model
2022/01/03 13:45:25 INFO mlflow.azureml: Registered an Azure Model with name: `wine_rf_model` and version: `1`

--------------------------------------------------------------------------- 
 WebserviceException Traceback (most recent call last)
 <command-3698963280378827> in <module> 
 2 from azureml . core . webservice import AciWebservice , Webservice
 3 
 ----> 4 registered_image, registered_model = mlflow.azureml.build_image(
 5 model_uri = "/mnt/gold/models/wine_rf_model" , 
 6 image_name = "wine_predict_image" , 

 /databricks/python/lib/python3.8/site-packages/mlflow/utils/annotations.py in deprecated_func (*args, **kwargs) 
 41 def deprecated_func ( * args , ** kwargs ) : 
 42 warnings . warn ( notice , category = DeprecationWarning , stacklevel = 2 ) 
 ---> 43 return func ( * args , ** kwargs ) 
 44 
 45 if func . __doc__ is not None : 

 /databricks/python/lib/python3.8/site-packages/mlflow/azureml/__init__.py in build_image (model_uri, workspace, image_name, model_name, mlflow_home, description, tags, synchronous) 
 222 tags = tags , 
 223 )
 --> 224 image = ContainerImage.create(
 225 workspace = workspace , 
 226 name = image_name , 

 /databricks/python/lib/python3.8/site-packages/azureml/core/image/image.py in create (workspace, name, models, image_config) 
 343 category=DeprecationWarning, stacklevel=2)
 344 
 --> 345 image_name_validation ( name ) 
 346 model_ids = Model . _resolve_to_model_ids ( workspace , models , name ) 
 347 

 /databricks/python/lib/python3.8/site-packages/azureml/_model_management/_util.py in image_name_validation (name) 
 1145 """
 1146 if not re . fullmatch ( r'^[a-z0-9]([-\.a-z0-9]*[a-z0-9])?$' , name ) or len ( name ) < 3 or len ( name ) > 32 : 
 -> 1147 raise WebserviceException('Error, provided image name is invalid. It must only consist of lowercase letters, '
 1148 'numbers, dashes or periods, start with a letter or number, end with a letter or ' 
 1149 'number, and be between 3 and 32 characters long.')

 WebserviceException : WebserviceException:
	Message: Error, provided image name is invalid. It must only consist of lowercase letters, numbers, dashes or periods, start with a letter or number, end with a letter or number, and be between 3 and 32 characters long.
	InnerException None
	ErrorResponse 
{
 "error": {
 "message": "Error, provided image name is invalid. It must only consist of lowercase letters, numbers, dashes or periods, start with a letter or number, end with a letter or number, and be between 3 and 32 characters long."
 }
}

DeprecationWarning: ``mlflow.azureml.build_image`` is deprecated since 1.19.0. This method will be removed in a near future release. Use ``the azureml deployment plugin, https://aka.ms/aml-mlflow-deploy`` instead.

In [0]:
from mlflow.deployments import get_deploy_client

# set the tracking uri as the deployment client
client = get_deploy_client(mlflow.get_tracking_uri())

# set the model path 
model_path = "model"

# define the model path and the name is the service name
# the model gets registered automatically and a name is autogenerated using the "name" parameter below 
client.create_deployment(model_uri='runs:/{}/{}'.format(runID, model_path),
                         name="wine-predict-aci")

Successfully registered model 'wine-predict-aci-model'.

--------------------------------------------------------------------------- 
 RestException Traceback (most recent call last)
 <command-3698963280378833> in <module> 
 9 # define the model path and the name is the service name 
 10 # the model gets registered automatically and a name is autogenerated using the "name" parameter below 
 ---> 11 client.create_deployment(model_uri='runs:/{}/{}'.format(runID, model_path),
 12 name="wine-predict-aci")

 /databricks/python/lib/python3.8/site-packages/azureml/mlflow/deploy/deployment_client.py in create_deployment (self, name, model_uri, flavor, config) 
 66 'AML currently only supports python_function.'.format(flavor))
 67 
 ---> 68 model_name , model_version = handle_model_uri ( model_uri , name ) 
 69 
 70 try : 

 /databricks/python/lib/python3.8/site-packages/azureml/mlflow/deploy/_util.py in handle_model_uri (model_uri, service_name) 
 77 # We will register the model for the user 
 78 model_name = service_name + "-model" 
 ---> 79 mlflow_model = mlflow_register_model ( model_uri , model_name ) 
 80 model_version = mlflow_model . version
 81 

 /databricks/python/lib/python3.8/site-packages/mlflow/tracking/_model_registry/fluent.py in register_model (model_uri, name, await_registration_for) 
 70 source = RunsArtifactRepository . get_underlying_uri ( model_uri ) 
 71 ( run_id , _ ) = RunsArtifactRepository . parse_runs_uri ( model_uri ) 
 ---> 72 create_version_response = client.create_model_version(
 73 name , source , run_id , await_creation_for = await_registration_for
 74 )

 /databricks/python/lib/python3.8/site-packages/mlflow/tracking/client.py in create_model_version (self, name, source, run_id, tags, run_link, description, await_creation_for) 
 2241 + " `source` field of the created model version. ===" 
 2242 )
 -> 2243 return self._get_registry_client().create_model_version(
 2244 name = name , 
 2245 source = new_source , 

 /databricks/python/lib/python3.8/site-packages/mlflow/tracking/_model_registry/client.py in create_model_version (self, name, source, run_id, tags, run_link, description, await_creation_for) 
 201 tags = tags if tags else { } 
 202 tags = [ ModelVersionTag ( key , str ( value ) ) for key , value in tags . items ( ) ] 
 --> 203 mv = self . store . create_model_version ( name , source , run_id , tags , run_link , description ) 
 204 if await_creation_for and await_creation_for > 0 : 
 205 _logger.info(

 /databricks/python/lib/python3.8/site-packages/azureml/mlflow/_internal/model_registry.py in create_model_version (self, name, source, *args, **kwargs) 
 48 source = 'azureml://artifacts/{}' . format ( artifacts_path ) 
 49 
 ---> 50 return super ( AzureMLflowModelRegistry , self ) . create_model_version ( name , source , * args , ** kwargs ) 

 /databricks/python/lib/python3.8/site-packages/mlflow/store/model_registry/rest_store.py in create_model_version (self, name, source, run_id, tags, run_link, description) 
 252 )
 253 )
 --> 254 response_proto = self . _call_endpoint ( CreateModelVersion , req_body ) 
 255 return ModelVersion . from_proto ( response_proto . model_version ) 
 256 

 /databricks/python/lib/python3.8/site-packages/azureml/mlflow/_internal/azureml_reststore.py in _call_endpoint (self, *args, **kwargs) 
 70 
 71 def _call_endpoint ( self , * args , ** kwargs ) : 
 ---> 72 return self . _call_endpoint_with_retries ( super ( AzureMLAbstractRestStore , self ) . _call_endpoint , * args , ** kwargs ) 
 73 
 74 @ classmethod

 /databricks/python/lib/python3.8/site-packages/azureml/mlflow/_internal/azureml_reststore.py in _call_endpoint_with_retries (cls, rest_store_call_endpoint, *args, **kwargs) 
 59 for i in range ( total_retry ) : 
 60 try : 
 ---> 61 return execute_func ( rest_store_call_endpoint , * args , ** kwargs ) 
 62 except RestException as rest_exception : 
 63 more_retries_left = i < total_retry - 1 

 /databricks/python/lib/python3.8/site-packages/azureml/_restclient/clientbase.py in execute_func (func, *args, **kwargs) 
 6